# Notebook de Extracción inicial (extraccionInicial.pynb)
### Este notebook aglutina todas la acciones realizadas en Python para la extracción inicial de los datos para la creación de un dataset inicial.

La información obtenida viene de dos fuentes principales:

1- **Base de datos myBestDR7**: Base de datos en SQL Server. Es una pequeña muestra de la base de datos general de SDSS en su versión DR7. Se utiliza esta muestra para obtener las galaxias que serán tratadas.

2- **Archivos Fits:** Archivos en formato FITS obtenidos de la API de la base de datos SDS. Se accederan a los archivos FITS correspondiente a las galaxias obtenidas de la base de datos myBestDR7.

En este notebook no se entraran en conceptos teóricos. Para ello acceder al notebook "Introducción a ficheros FITS"



In [1]:
# Preparación
# Llamamos a las librerías que serán necesarias en la ejecución de este notebook
from astropy.io import fits  # Tratamiento de ficheros Fits
import pandas as pd  # Tratamiento de datos como tablas
import os # Librería para gestionar Sisitema Operativo

# Cambiamos la carpeta de trabajo a dónde tenemos los archivos
os.chdir("/home/vicente/Galassify/NoteBooks")

# Configuramos la forma en que se muestran los float (4 decimales)
pd.options.display.float_format = "{:20,.4f}".format


## Utilización de la API para acceso a archivos Fits
Se puede acceder y bajar cualquier archivo fits correspondiente a un espectro determinado, siempre y cuando se tengan las referencias de plate, mjs y fiber. 

También se puede acceder al archivo en formato fit o csv, así como una versión reducida o completa. 

Para ello utilizaremos la siguiente API:

`https://dr14.sdss.org/optical/spectrum/view/data/format=FORMAT/spec=SPEC?plateid=PLATEID&mjd=MJD&fiberid=FIBERID`<br>
Ejemplo: Para bajar el siguiente fichero FITS:<br>
SURVEY : dr14<br>
FORMAT : fits<br>
SIZE : little<br>
PLATEID : 0285<br>
MJD : 51930<br>
FIBERID : 0184<br>
Llamariamos a https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=0285&mjd=51930&fiberid=0184

In [2]:
# Para facilitar la creación del datset incial, vamos a crear un script que pasado los parametros plate, mjd y fiber
# baje el archivo fit utilizando la API de SDS explicada y devuelve un dataframe con los datos cargados
#Utilizaremos la data release 14

def cargarFit(plate, mjd, fiber):

    url = 'https://dr14.sdss.org/optical/spectrum/view/data/format=fits/spec=lite?plateid=' + str(plate) + '&mjd=' + str(mjd) + '&fiberid=' + str(fiber)
    try:
        hdul = fits.open(url, cache=False)
    except:
        print("**** ERROR bajando archivo ", url)
        return 0
    
    z = hdul[2].data['Z'][0]
    flux = hdul[1].data['FLUX']
    loglam = hdul[1].data['LOGLAM']

    df = pd.DataFrame(columns=['Plate', 'MJD', 'FIBER', 'Z', 'FLUX', 'LOGLAM'])

    # Rellenamos el Dataframe
    for i in range(len(flux)):
        campos = list([plate, mjd, fiber, z, flux[i], loglam[i]])
        registrodf = pd.DataFrame([campos], columns = list(df.columns))
        df = df.append(registrodf, ignore_index=True)

    # Calculamos la columna de longitud de onda en Amstrongs. Recordamos que el valor que se encuentra en el archivo fit es
    # el logaritmo en base 10
    df['LAM'] = 10**df['LOGLAM']

    # Calculamos la corrección del redshift aplicando el factor de corrección Z
    df['LAMZ'] = df['LAM']/(1+df['Z'])
    
    return df

## DataSet Base con la relación inicial de Galaxias
El archivo de entrada dónde se relacionan las galaxias cuyos archivos FIT hay que bajar, se obtiene de realizar la siguientes consulta en la SDSS

SELECT top 3000 Q.plate, Q.mjd, Q.fiberid from
(
SELECT TOP 100000 NEWID() as R, s.plate, s.mjd, s.fiberid
FROM SpecObj AS s
where (s.z < 0.25) and (s.class = 'GALAXY')
) as Q
Order by Q.R

En esta consulta se obtienen un total de 3000 Galaxias (top 3000) de manera aleatoria (NEWID).

Esta consulta hay que exportarla a un fichero cvs que servirá como fichero de entrada. 
La web dónde se puede realizar la consulta es la siguiente:
http://skyserver.sdss.org/dr15/en/tools/search/sql.aspx

## Generación de campos adicionales
Vamos a proceder a la extracción de los espectros buscados en nuestro Dataset, pero debemos generar campos adicionales que nos permitan realizar la corrección del desaplazamiento al rojo.

Los campos que extreremos de los archivos FITS serán:
* **Plate:** Plancha desde dónde se realizó la observación del espectro
* **MJD:** Fecha Juliana de la observación
* **FIBER:** Fibra desde dónde se realizó la observación del espectro
* **Z:** Factor Z de corrección del espectro
* **FLUX:** Valor de luminancia de la observación en la frecuencia dada
* **LOGLAM** Logaritmo en base 10 de la frecuencia de la ue¡muestra en Amstrongs

Adicionalmente tendremos que calcucar los siguientes campos
* **LAM:** Frecuencia de la muestra en Amstrong (sin el logaritmo)
* **LAMZ:** Frecuencia de la muestra en Amstring con la corrección del redshift aplicada (LAMZ = LAM/(1+Z))

In [8]:
csvEntrada = "muestraGalaxias3KRandom.csv"
csvSalida = "datasetV1_8Campos3K.csv"

# Cargamos el dataset donde tenemos la lista de galaxias en terminos de Plate, MJD y Fiber
dfMuestra = pd.read_csv(csvEntrada, sep=';')

# Comprobamos que no hay ningun NaN en el dataset
dfMuestra.isnull().any()

PLATE    False
MJD      False
FIBER    False
dtype: bool

In [9]:
# Creamos el dataset que irá contruyendo en el csv de salida.
# ******************** ATENCION Sólo ejecutar si se quiere empezar desde 0 **************************

datasetV1 = pd.DataFrame(columns=['PLATE', 'MJD', 'FIBER', 'Z', 'FLUX', 'LOGLAM', 'LAM', 'LAMZ'])

datasetV1.to_csv(csvSalida, index=False, header=True, sep=';')


In [10]:
import os

# Calculamos el numero de espectros que ya hay en el csv de Salida para continuar con el proceso.
if (os.path.isfile(csvSalida)):
    # Iteramos sobre el dataset de muestra para ir construyendo el DataSet principal
    csv = pd.read_csv(csvSalida, sep=';')
    ultimaFila = csv[['PLATE', 'MJD', 'FIBER']].drop_duplicates().shape[0]
    
    for i in range(ultimaFila, len(dfMuestra)):
        plate = str(dfMuestra.loc[i, 'PLATE'])
        MJD = str(dfMuestra.loc[i, 'MJD'])
        fiber = str(dfMuestra.loc[i, 'FIBER'])
        df = 0
        while(type(df) == int):
            df = cargarFit(plate, MJD, fiber)
        df.to_csv(csvSalida, mode='a', index=False, header=False, sep=";")
        # datasetV1 = datasetV1.append(df, ignore_index=True)
        print (str(i) + ' (' + str(dfMuestra.loc[i, 'PLATE']) + ', ' + str(dfMuestra.loc[i, 'MJD']) + ', ' + str(dfMuestra.loc[i, 'FIBER']) + ') -> Hecho!')
else:
    print("El archivo " , csvSalida , "no existe")

    


0 (2099, 53469, 148) -> Hecho!
1 (1815, 53884, 1) -> Hecho!
2 (699, 52202, 92) -> Hecho!
3 (1924, 53330, 267) -> Hecho!
4 (5440, 55983, 132) -> Hecho!
5 (6035, 56076, 28) -> Hecho!
6 (687, 52518, 140) -> Hecho!
7 (2595, 54207, 561) -> Hecho!
8 (861, 52318, 183) -> Hecho!
9 (512, 51992, 220) -> Hecho!
10 (2291, 53714, 166) -> Hecho!
11 (449, 51900, 390) -> Hecho!
12 (2956, 54525, 616) -> Hecho!
13 (1972, 53466, 615) -> Hecho!
14 (758, 52253, 290) -> Hecho!
15 (7841, 56960, 488) -> Hecho!
16 (600, 52317, 139) -> Hecho!
17 (2238, 54205, 99) -> Hecho!
18 (955, 52409, 261) -> Hecho!
19 (826, 52295, 68) -> Hecho!
20 (1486, 52993, 57) -> Hecho!
21 (1357, 53034, 299) -> Hecho!
22 (788, 52338, 209) -> Hecho!
23 (1288, 52731, 125) -> Hecho!
24 (4998, 55722, 248) -> Hecho!
25 (481, 51908, 114) -> Hecho!
26 (2421, 54153, 382) -> Hecho!
27 (508, 52366, 356) -> Hecho!
28 (4263, 55484, 590) -> Hecho!
29 (779, 52342, 63) -> Hecho!
30 (1270, 52991, 51) -> Hecho!
31 (745, 52258, 118) -> Hecho!
32 (2245,